# Hyperparameter Tuning And Regularization

We ended the last video with a classification accuracy of 77%. However, there are a few more nobs we can turn to improve the performance.

Our growing set of imports

In [ ]:
import os

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import LeaveOneGroupOut

import activity_classifier_utils

## Load Data and Extract Features

In [ ]:
fs = 256
data = activity_classifier_utils.LoadWristPPGDataset()
labels, subjects, features = activity_classifier_utils.GenerateFeatures(data,
                                                                        fs,
                                                                        window_length_s=10,
                                                                        window_shift_s=10)

## Hyperparameter Tuning

### Define hyperparameters

How many hyperparameters should we really use and how big should these trees be. At first we made our best guesses, but now we can explore this space and see if the performance changes.

In [ ]:
n_estimators_opt = [2, 10, 20, 50, 100, 150, 300]
max_tree_depth_opt = range(2, 7)

In [ ]:
class_names = np.array(['bike', 'run', 'walk'])
logo = LeaveOneGroupOut()
accuracy_table = []

In [ ]:
import itertools

for n_estimators, max_tree_depth in itertools.product(n_estimators_opt, max_tree_depth_opt):
    # Iterate over each pair of hyperparameters
    cm = np.zeros((3, 3), dtype='int')                       # Create a new confusion matrix
    clf = RandomForestClassifier(n_estimators=n_estimators,  # and a new classifier  for each
                                 max_depth=max_tree_depth,   # pair of hyperparameters
                                 random_state=42,
                                 class_weight='balanced')
    for train_ind, test_ind in logo.split(features, labels, subjects):
        # Do leave-one-subject-out cross validation as before.
        X_train, y_train = features[train_ind], labels[train_ind]
        X_test, y_test = features[test_ind], labels[test_ind]
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        c = confusion_matrix(y_test, y_pred, labels=class_names)
        cm += c
    # For each pair of hyperparameters, compute the classification accuracy
    classification_accuracy = np.sum(np.diag(cm)) / np.sum(np.sum(cm))
    
    # Store the hyperparameters and the classification accuracy that resulted
    # from the model created with them.
    accuracy_table.append((n_estimators, max_tree_depth, classification_accuracy))

In [ ]:
accuracy_table_df = pd.DataFrame(accuracy_table,
                                 columns=['n_estimators', 'max_tree_depth', 'accuracy'])

In [ ]:
accuracy_table_df.head()

In [ ]:
accuracy_table_df.loc[accuracy_table_df.accuracy.idxmax()]

Just by reducing the maximum tree depth to 2, we have significantly increased our classification accuracy, from 77% to 89%. By reducing the depth to 2, we are **regularizing** our model. Regularization is an important topic in ML and is our best way to avoid overfitting. This is why we see an increase in the cross-validated performance.

But, we used the entire dataset many times to figure out the optimal hyperparameters. In some sense, this is also overfitting. Our 90% classification accuracy is likely too high, and not the generalized performance. In the next video, we can see what our actual generalized performance might be if we use our dataset to optimize hyperparameters

## Nested Cross Validation

To get a more accurate idea of the performance, we'd ideally pick the best hyperparameters on a subset of the data, and then evaluate it on a hold-out set. This is similar to a train-validation-test set split. When you don't have enough data to separate your dataset into 3 parts, we can nest the hyperparameter selection in another layer of cross-validation.

Be patient, this takes a while. On my machine -- 3.3 GHz Intel Core i7 on a MacBook Pro 2016 -- it took less than 8 minutes.

In [ ]:
class_names = ['bike', 'run', 'walk']

# Store the confusion matrix for the outer CV fold.
nested_cv_cm = np.zeros((3, 3), dtype='int')
splits = 0

for train_val_ind, test_ind in logo.split(features, labels, subjects):
    # Split the dataset into a test set and a training + validation set.
    # Model parameters (the random forest tree nodes) will be trained on the training set.
    # Hyperparameters (how many trees and the max depth) will be trained on the validation set.
    # Generalization error will be computed on the test set.
    X_train_val, y_train_val = features[train_val_ind], labels[train_val_ind]
    subjects_train_val = subjects[train_val_ind]
    X_test, y_test = features[test_ind], labels[test_ind]
    
    # Keep track of the best hyperparameters for this training + validation set.
    best_hyper_parames = None
    best_accuracy = 0
    
    for n_estimators, max_tree_depth in itertools.product(n_estimators_opt,
                                                          max_tree_depth_opt):
        # Optimize hyperparameters as above.
        inner_cm = np.zeros((3, 3), dtype='int')
        clf = RandomForestClassifier(n_estimators=n_estimators,
                                     max_depth=max_tree_depth,
                                     random_state=42,
                                     class_weight='balanced')
        for train_ind, validation_ind in logo.split(X_train_val, y_train_val,
                                                    subjects_train_val):
            X_train, y_train = X_train_val[train_ind], y_train_val[train_ind]
            X_val, y_val = X_train_val[validation_ind], y_train_val[validation_ind]
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_val)
            c = confusion_matrix(y_val, y_pred, labels=class_names)
            inner_cm += c
        classification_accuracy = np.sum(np.diag(inner_cm)) / np.sum(np.sum((inner_cm)))
        
        # Keep track of the best pair of hyperparameters.
        if classification_accuracy > best_accuracy:
            best_accuracy = classification_accuracy
            best_hyper_params = (n_estimators, max_tree_depth)
    
    # Create a model with the best pair of hyperparameters for this training + validation set.
    best_clf = RandomForestClassifier(n_estimators=best_hyper_params[0],
                                      max_depth=best_hyper_params[1],
                                      class_weight='balanced')
    
    # Finally, train this model and test it on the test set.
    best_clf.fit(X_train_val, y_train_val)
    y_pred = best_clf.predict(X_test)
    
    # Aggregate confusion matrices for each CV fold.
    c = confusion_matrix(y_test, y_pred, labels=class_names)
    nested_cv_cm += c
    splits += 1
    print('Done split {}'.format(splits))


Notice the drop in performance. This is because we are now not overfitting our hyperparameters when we evaluate model performance.

In [ ]:
np.sum(np.diag(nested_cv_cm)) / np.sum(np.sum(nested_cv_cm))

## Feature Importance

Another way to regularize our model and increase performance (besides reducing the tree depth) is to reduce the number of features we use.

The `RandomForestClassifier` can tell us how important the features are in classifying the data.

In [ ]:
clf = RandomForestClassifier(n_estimators=100,
                             max_depth=4,
                             random_state=42,
                             class_weight='balanced')
activity_classifier_utils.LOSOCVPerformance(features, labels, subjects, clf)
clf.feature_importances_

Let's see what the 10 most important features are.

In [ ]:
sorted(list(zip(clf.feature_importances_, activity_classifier_utils.FeatureNames())), reverse=True)[:10]

Let's train our original model on just the 10 best features as determined by the `RandomForestClassifier`

In [ ]:
sorted_features = sorted(zip(clf.feature_importances_, np.arange(len(clf.feature_importances_))), reverse=True)
best_feature_indices = list(zip(*sorted_features))[1]
X = features[:, best_feature_indices[:10]]

In [ ]:
X.shape

In [ ]:
cm = activity_classifier_utils.LOSOCVPerformance(X, labels, subjects, clf)
activity_classifier_utils.PlotConfusionMatrix(cm, class_names, normalize=True)
print('Classification accuracy = {:0.2f}'.format(np.sum(np.diag(cm)) / np.sum(np.sum(cm))))

We no longer misclassify `bike` as `walk`. We've improved our classifier performance by 15%, just by picking the most important features! 